#### Jorge Castro DAPT BER

<div class="alert alert-block alert-info">



# Lab (Customer Analysis Final Round )



Jump to:

* [01 - Problem (case study)](#01---Problem-(case-study))
    * [Data Description](#Data-Description)
    * [Goal](#Goal)    
* [02 - Getting Data](#02---Getting-Data)
    * [Read the .csv file](#Read-the-.csv-file)    
* [03 - Cleaning/Wrangling/EDA](#03---Cleaning/Wrangling/EDA)
    * [Change headers names](#Change-headers-names)
    * [Deal with NaN values](#Deal-with-NaN-values)
    * [Categorical Features](#Categorical-Features)
    * [Numerical Features](#Numerical-Features)
    * Exploration    
* [04 - Processing Data](#04---Processing-Data)
    * [Dealing with outliers](#Dealing-with-outliers)
    * [Normalization](#Normalization)
    * [Encoding Categorical Data](#Encoding-Categorical-Data)
    * [Splitting into train set and test set](#Splitting-into-train-set-and-test-set)    
* [05 - Modeling](#05---Modeling)
    * Apply model
* [06 - Model Validation](#06---Model-Validation)
    * [R2](#R2)
    * [MSE](#MSE)
    * [RMSE](#RMSE)
    * [MAE](#MAE)
* [07 - Reporting](#07---Reporting)
    * [Present results](#Present-results)


</div>

[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)


print(link_title)

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import sweetviz as sv
%matplotlib inline

# 01 - Problem (case study)
[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)

##### Data Description

    


Dataset from an insurance company

##### Goal
[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)

To run predictive analytics to analyze the most profitable customers and how they interact.

# 02 - Getting Data
##### Read the .csv file
[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)


In [2]:
df_d = pd.read_csv('marketing_customer_analysis.csv')
# Showing dataframe shape

df_d.shape

(10910, 26)

In [3]:
df_d.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,Location Code,Marital Status,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,48029,Suburban,Married,61,7.0,52,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,0,Suburban,Single,64,3.0,26,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,22139,Suburban,Single,100,34.0,31,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,49078,Suburban,Single,97,10.0,3,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,23675,Suburban,Married,117,NaN,31,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


# 03 - Cleaning/Wrangling/EDA
[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)

##### Change headers names

In [4]:
# Lower caseing headers
df_d.columns = df_d.columns.str.lower()

In [5]:
# Replacing spaces by underscores
df_d.columns = df_d.columns.str.replace(' ', '_')

In [6]:
# Dropping the column 'unnamed:_0' as it is a duplicate from the index
df_d = df_d.drop('unnamed:_0', 1)

##### Deal with NaN values
[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)

In [7]:
# Dealing with null values:


# We have to create a paralelle DataFrame but only indicating the null % of each series,
# then we use it as a guide to drop the series that have more than n% of nulls
# we save on a variable the creation of the dataframe with our calcule of null %

nulls_df = pd.DataFrame(round(df_d.isna().sum()/len(df_d)*100,2))

# We need to rename the headers so we can perform the column drop based on null %, 
# but first we need to reset the index as this 2 dataframes have 
# different lenghts (amount of rows)

# Reseting index
nulls_df = nulls_df.reset_index()

# Renaming headers of our new mini DataFrame to read null %
nulls_df.columns = ['header_name', '%_nulls']


# saving our calculations for the drop of headers on a variable based on % nulls
columns_drop = nulls_df[nulls_df['%_nulls'] > 50 ]['header_name']

# Dropping columns using the variable I just saved columns_drop
df1 = df_d.drop(columns_drop, axis = 1)

# Reseting the index after the drop of values
df1.reset_index()

# to address the rest of the null values, 
# which have a low % of nulls for each series, I will use the 
# function dropna which has a parametre called 'how' and the arguments are any or all
# ANY means if there is at least 1 null in ANY row, the row will be droped

# Creating a DataFrame to visualize the remaining nulls
nulls_percent_2 = pd.DataFrame(round(df1.isna().sum()/len(df1)*100,2))





In [8]:
# Lets have a look at the remaining % of nulls

nulls_percent_2

,0
customer,0.00
state,5.78
customer_lifetime_value,0.00
response,5.78
coverage,0.00
education,0.00
effective_to_date,0.00
employmentstatus,0.00
gender,0.00
income,0.00


In [9]:
# Applying function "dropna" which has a parametre called 'how' and the 
# arguments are "any" or "all"
# "any" means if there is at least 1 null in ANY row, the row will be droped

df = df1.dropna(how='any')

In [10]:
# Verifying if all the nulls have been erradicated 
nulls_percent_3 = pd.DataFrame(round(df.isna().sum()/len(df)*100,2))

In [11]:
nulls_percent_3

,0
customer,0.0
state,0.0
customer_lifetime_value,0.0
response,0.0
coverage,0.0
education,0.0
effective_to_date,0.0
employmentstatus,0.0
gender,0.0
income,0.0


In [12]:
df.head()

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,location_code,marital_status,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,48029,Suburban,Married,61,7.0,52,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize
1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,0,Suburban,Single,64,3.0,26,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize
2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,22139,Suburban,Single,100,34.0,31,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize
3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,49078,Suburban,Single,97,10.0,3,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize
6,IW72280,California,5035.035257,No,Basic,Doctor,2/14/11,Employed,F,37405,Urban,Married,63,8.0,99,3.0,4,Corporate Auto,Corporate L2,Offer2,Branch,287.556107,Four-Door Car,Medsize


In [13]:
# Rounding decimals in columns: 2 and 21
df.iloc[:, [2,21]]


,customer_lifetime_value,total_claim_amount
0,4809.216960,292.800000
1,2228.525238,744.924331
2,14947.917300,480.000000
3,22332.439460,484.013411
6,5035.035257,287.556107
...,...,...
10903,2771.663013,355.200000
10904,19228.463620,897.600000
10906,5259.444853,273.018929
10907,23893.304100,381.306996


In [14]:
# Rounding decimals: 
df = pd.DataFrame(df.round({'customer_lifetime_value': 2, 'total_claim_amount': 2}))

In [15]:
# Creating a new DataFrame with columns that are important to keep
df_0 = pd.DataFrame(df.iloc[:, [1, 2, 5, 7, 8, 9, 11, 16, 21]])

In [16]:
df_0.head()

,state,customer_lifetime_value,education,employmentstatus,gender,income,marital_status,number_of_policies,total_claim_amount
0,Arizona,4809.22,College,Employed,M,48029,Married,9,292.80
1,California,2228.53,College,Unemployed,F,0,Single,1,744.92
2,Washington,14947.92,Bachelor,Employed,M,22139,Single,2,480.00
3,Oregon,22332.44,College,Employed,M,49078,Single,2,484.01
6,California,5035.04,Doctor,Employed,F,37405,Married,4,287.56


##### Categorical Features
[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)

In [17]:
df_cat = pd.DataFrame(df_0.select_dtypes(include='object'))

In [18]:
df_cat

,state,education,employmentstatus,gender,marital_status
0,Arizona,College,Employed,M,Married
1,California,College,Unemployed,F,Single
2,Washington,Bachelor,Employed,M,Single
3,Oregon,College,Employed,M,Single
6,California,Doctor,Employed,F,Married
...,...,...,...,...,...
10903,Arizona,College,Employed,M,Married
10904,Nevada,High School or Below,Unemployed,M,Single
10906,Oregon,College,Employed,F,Married
10907,Arizona,Bachelor,Employed,F,Married


##### Numerical Features
[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)

In [19]:
# # To see only the colums with numeric datatypes we use the get_numeric_data method.

In [20]:
df_num = pd.DataFrame(df_0._get_numeric_data())

In [21]:
df_num

,customer_lifetime_value,income,number_of_policies,total_claim_amount
0,4809.22,48029,9,292.80
1,2228.53,0,1,744.92
2,14947.92,22139,2,480.00
3,22332.44,49078,2,484.01
6,5035.04,37405,4,287.56
...,...,...,...,...
10903,2771.66,59855,1,355.20
10904,19228.46,0,2,897.60
10906,5259.44,61146,6,273.02
10907,23893.30,39837,2,381.31


##### Exploration
[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)

# 04 - Processing Data
[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)

##### Dealing with outliers
[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)

##### Normalization
[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)



##### Encoding Categorical Data
[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)


##### Splitting into train set and test set
[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)


# 05 - Modeling
##### Apply model
[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)

# 06 - Model Validation
[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)

##### R2

##### MSE

##### RMSE

##### MAE

# 07 - Reporting
##### Present results
[Back to Top](#Jorge-Castro-DAPT-BER)

[Go down](#Down)


[Back to Top](#Jorge-Castro-DAPT-BER)

<div class="alert alert-block alert-info">
<b>Tip:</b> Use blue boxes (alert-info) for tips and notes. 
If it’s a note, you don’t have to include the word “Note”.
</div>

<div class="alert alert-block alert-warning">
<b>Example:</b> Use yellow boxes for examples that are not 
inside code cells, or use for mathematical formulas if needed.
</div>

<div class="alert alert-block alert-success">
<b>Up to you:</b> Use green boxes sparingly, and only for some specific 
purpose that the other boxes can't cover. For example, if you have a lot 
of related content to link to, maybe you decide to use green boxes for 
related links from each section of a notebook.
</div>

<div class="alert alert-block alert-danger">
<b>Just don't:</b> In general, avoid the red boxes. These should only be
used for actions that might cause data loss or another major issue.
</div>

[Back to Top](#Jorge-Castro-DAPT-BER)

##### Down

# Conclusion of analysis findings